In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import cupy as cp

In [5]:
PROJECT_PATH = '../'
BLUE_PATH = "/scratch/ceph/szalay/swei20/AE/norm_flux_Blue_R2300.h5"
NIR_PATH = "/scratch/ceph/szalay/swei20/AE/norm_flux_NIR_R4300.h5"
sys.path.insert(0, PROJECT_PATH)

# Red Arm

In [7]:
from lv.base.basePCA import PCA

In [23]:
p=PCA()
p.W = p.Ws["RedM"]

In [13]:
PATH = f"/scratch/ceph/szalay/swei20/AE/norm_flux_{p.W[3]}_R{p.W[2]}.h5"

with h5py.File(PATH, 'r') as f:
    flux0 = f['flux'][()]
    wave0 = f['wave'][()]
    para0 = f['para'][()]

In [25]:
p.prepare_data(flux0, wave0, para0)

# M flux: 7689, wave None: (2204,) 
# W flux: 6480, wave None: (2204,) 
# C flux: 3780, wave None: (2204,) 
# B flux: 3300, wave None: (2204,) 
# R flux: 2100, wave None: (2204,) 
# G flux: 9972, wave None: (2204,) 


In [42]:
p.pcFlux={}
p.npcFlux={}

In [43]:
p.prepare_svd()

100%|██████████| 6/6 [00:47<00:00,  7.88s/it]


In [44]:
SPATH = f"/scratch/ceph/swei20/data/dnn/pc/bosz_{p.W[3]}_R{p.W[2]}.h5"

In [45]:
p.save_PCA(SPATH)

In [31]:
p.pcs["M"].shape

(7689, 200)

In [34]:
p.dfpara

,F,T,L,C,O
0,-2.5,3500.0,0.0,-0.50,-0.25
1,-2.5,3500.0,0.0,-0.50,0.00
2,-2.5,3500.0,0.0,-0.50,0.25
3,-2.5,3500.0,0.0,-0.50,0.50
4,-2.5,3500.0,0.0,-0.25,-0.25
...,...,...,...,...,...
51607,-1.0,5500.0,3.5,0.25,0.50
51608,-1.0,5500.0,3.5,0.50,-0.25
51609,-1.0,5500.0,3.5,0.50,0.00
51610,-1.0,5500.0,3.5,0.50,0.25


# Blue Arm

In [6]:
from lv.base.basePCA import PCA

In [7]:
p=PCA()
p.W = p.Ws["Blue"]

In [8]:
PATH = f"/scratch/ceph/szalay/swei20/AE/norm_flux_{p.W[3]}_R{p.W[2]}.h5"

with h5py.File(PATH, 'r') as f:
    flux0 = f['flux'][()]
    wave0 = f['wave'][()]
    para0 = f['para'][()]

In [9]:
p.prepare_data(flux0, wave0, para0)

# M flux: 7689, wave None: (5368,) 
# W flux: 6480, wave None: (5368,) 
# C flux: 3780, wave None: (5368,) 
# B flux: 3300, wave None: (5368,) 
# R flux: 2100, wave None: (5368,) 
# G flux: 9972, wave None: (5368,) 


In [11]:
p.prepare_svd()

100%|██████████| 6/6 [00:45<00:00,  7.52s/it]


In [12]:
SPATH = f"/scratch/ceph/swei20/data/dnn/pc/bosz_{p.W[3]}_R{p.W[2]}.h5"

In [13]:
p.save_PCA(SPATH)